# fazy

Tugas kedua Mata Kuliah Pengantar Kecerdasan buatan pada Semester 5

Ditulis oleh Jordi Yaputra (1301180353)

## Imports

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

## Utilitas

In [ ]:
class util:
    @staticmethod
    def clamp(x, a, b):
        if type(x) is np.ndarray or type(b) is np.ndarray:
            return np.clip(x, a, b)
        return max(a, min(x, b))

read_excel = pd.read_excel

## Fazy

In [ ]:
# %% class set fungsi anggota
class FungSet:
  def __init__ (self, label, fungsi, bilangan:tuple):
    prev = bilangan[0]
    for bil in bilangan[1:]:
      assert prev <= bil, 'Bilangan harus terurut'
      prev = bil

    self.label = label
    self.fungsi = fungsi
    self.bilangan = bilangan

  def hitung(self, x, up=1):
    return self.fungsi(x, *self.bilangan, up=up)

In [ ]:
# %% fungsi keanggotaan
class Fang:

  @staticmethod
  def linier_atas(x, a, b, up=1):
    return util.clamp((x - a) / (b - a), 0, up)

  @staticmethod
  def linier_bawah(x, a, b, up=1):
    return util.clamp((b - x) / (b - a), 0, up)

  @staticmethod
  def segitiga(x, a, b, c, up=1):
    if type(x) is np.ndarray:
      arr = np.where(x <= b,
        Fang.linier_atas(x, a, b, up),
        Fang.linier_bawah(x, b, c, up))
      return arr
    elif x <= b:
      return Fang.linier_atas(x, a, b, up)
    return Fang.linier_bawah(x, b, c, up)

  @staticmethod
  def trapesium(x, a, b, c, d, up=1):
    if type(x) is np.ndarray:
      arr = np.where(x <= c,
          Fang.linier_atas(x, a, b, up),
          Fang.linier_bawah(x, c, d, up))
      return arr
    elif x <= c:
      return Fang.linier_atas(x, a, b, up)
    return Fang.linier_bawah(x, c, d, up)

In [ ]:
# %% class utamanya
class Fazy:
  def __init__(self, arr_fset:tuple, lookup_inferensi:pd.core.frame.DataFrame):
    # definisi notasi:
    # m     : banyaknya data (baris) di dataset
    # n     : banyaknya rules inferensi
    # c     : banyaknya kolom inferensi
    # nfs_i : banyaknya fungset di arr_fset di kolom ke-i

    for fset_tup in arr_fset:
      for fs in fset_tup:
        assert type(fs) == FungSet, \
          'arr_fset harus merupakan tuple berisi tuple FungSet'

    self.arr_fset = arr_fset

    # nama-nama kolom di inferensi
    self.cols = [col for col in lookup_inferensi]

    # one hot encoding setiap kategori di setiap kolom inferensi
    self.one_hot = tuple(
        pd.get_dummies(lookup_inferensi[col])[\
          [f.label for f in arr_fset[i]]] \
        for i, col in enumerate(self.cols))

  def _fazify(self, nilai:np.ndarray, fset_tup:tuple):
    return np.array([fs.hitung(nilai) for fs in fset_tup])

  def _inferensi(self, fazys):
    masked = tuple(
      np.dot(self.one_hot[i], fazys[i]) for i in range(len(fazys))
    )
    konjungsi = np.minimum(*masked)  # shape = (n, m)
    # matriks disjungsi
    disjungsi = np.dot(self.one_hot[-1].T, konjungsi) # shape = (nfs_out, m)
    return disjungsi

  def _defazify(self, inferensi, step, maks, mins):
    sumbu_x = np.arange(mins, maks, step=step)
    derajat = np.max([
      [fs.hitung(x, up=inferensi[i]) for x in sumbu_x] \
      for i, fs in enumerate(self.arr_fset[-1])
      ], axis=0)
    return np.dot(sumbu_x, derajat) / np.sum(derajat, axis=0)

  def klasify(self, dataset: pd.core.frame.DataFrame, step=10, maks=100, mins=0):
    fazys = tuple(  # fazys_i.shape = (nfs_i, m)
        self._fazify(dataset[self.cols[i]].to_numpy(), fset_tup) \
        for i, fset_tup in enumerate(self.arr_fset[:len(self.arr_fset)-1]))
    inferensi = self._inferensi(fazys)
    return self._defazify(inferensi, step, maks, mins)

## Main

In [ ]:
# %% membaca data
data_mhs = read_excel('data/Mahasiswa.xls')
data_mhs.head()

In [ ]:
# %% visualisasi data raw
data_mhs['Selisih'] = data_mhs['Penghasilan'] - data_mhs['Pengeluaran']
plt.xlabel('Penghasilan')
plt.ylabel('Pengeluaran')
plt.grid(True)
sct = plt.scatter(data_mhs['Penghasilan'], data_mhs['Pengeluaran'],
                  marker='o',
                  c=(data_mhs['Selisih'] < 0))
plt.legend(handles=sct.legend_elements()[0],
           labels=['Selisih positif', 'Selisih negatif'],
           loc='lower right')

In [ ]:
# %% knowledge base
set_fungsi_penghasilan = (
  FungSet('kecil', Fang.linier_bawah, (4, 8)),
  FungSet('sedang', Fang.segitiga, (8, 9, 11)),
  FungSet('besar', Fang.linier_atas, (10, 20))
)

set_fungsi_pengeluaran = (
  FungSet('kecil', Fang.linier_bawah, (3, 6)),
  FungSet('sedang', Fang.segitiga, (5, 8, 10)),
  FungSet('besar', Fang.linier_atas, (9, 11)),
)

set_fungsi_kelayakan = (
  FungSet('rendah', Fang.linier_bawah, (20, 50)),
  FungSet('tinggi', Fang.linier_atas, (30, 60))
)

In [ ]:
# %% visualisasi fungset
n_20 = np.arange(20, step=.1);
n_100 = np.arange(100, step=5);
h_pengh = np.max([fs.hitung(n_20) for fs in set_fungsi_penghasilan], axis=0)
h_penge = np.max([fs.hitung(n_20) for fs in set_fungsi_pengeluaran], axis=0)
h_kely = np.max([fs.hitung(n_100) for fs in set_fungsi_kelayakan], axis=0)

ylim = (0, 1.2)
plt.subplots_adjust(hspace=.4, top=1.5)
plt.subplot(3, 1, 1)
plt.ylim(ylim)
plt.title('Fungset Penghasilan')
plt.plot(n_20, h_pengh)
plt.subplot(3, 1, 2)
plt.ylim(ylim)
plt.title('Fungset Pengeluaran')
plt.plot(n_20, h_penge)
plt.subplot(3, 1, 3)
plt.ylim(ylim)
plt.title('Fungset Kelayakan')
plt.plot(n_100, h_kely)

In [ ]:
#%% tabel inferensi
lookup_inferensi = pd.DataFrame({
  'Penghasilan': (
      'kecil', 'kecil', 'kecil',
      'sedang','sedang','sedang',
      'besar','besar','besar'),
  'Pengeluaran': (
      'kecil','sedang','besar',
      'kecil','sedang','besar',
      'kecil','sedang','besar'),
  'Kelayakan': (
      'rendah','tinggi','tinggi',
      'rendah','rendah','tinggi',
      'rendah','rendah','rendah')
  })

print('Tabel aturan inferensi')
lookup_inferensi

In [ ]:
# %% main
model = Fazy((
    set_fungsi_penghasilan,
    set_fungsi_pengeluaran,
    set_fungsi_kelayakan
  ),
  lookup_inferensi=lookup_inferensi)

kelayakan = model.klasify(dataset=data_mhs, step=5) # (2, m)

In [ ]:
# %% visualisasi tabel
data_mhs['Kelayakan'] = kelayakan
df1 = data_mhs.sort_values('Kelayakan', ascending=False)[:20]
# df2 = data_mhs.sort_values('Selisih', ascending=True)[:20]
print('Tabel terpilih terurut berdasarkan kelayakan')
display(df1)
# display(df2)

In [ ]:
# %% output ke excel
with pd.ExcelWriter('data/Bantuan.xls') as writer:
  df1.to_excel(writer, 'data_bantuan', columns=['Id'], index=False, header=False)

In [ ]:
# %% uji korelasi
corr = data_mhs.loc[:, ['Kelayakan', 'Selisih']].corr().to_numpy()[0,1]
print('Korelasi selisih-kelayakan', corr, sep='\n')